05 - TENSORFLOW TUTORIAL - Placeholders
===========================================

In [1]:
# ==============================================================================
#                                                                        IMPORTS
# ==============================================================================
from __future__ import print_function, division
import tensorflow as tf
import numpy as np

Placeholders can be thought of as containers. They contain nothing on 
their own untill you fill them up. And just like containers, there are 
also restrictions about the shape and size of things you can put in it. 

Placeholders are useful when you have tensors whose values you want to 
control, and which you expect to be different on separate sessions. 
So for neural networks, you would want to use placeholders for the 
input vectors/matrices, since you will have a different of values for 
the training set, validation set, test set, and the inputs you use 
as your final product. 


## Declaring a placeholder 

Declaring a placeholder is just a matter of placing a line like 
the following when creating a graph. 

```python
tf.placeholder(tf.float32, shape=[3,4])
```

Here we declared that we want a tensor that is 3 by 4 in size 
and composed of elements that are or can be converted to 32 
bit floats. 

## Feed me! I'm hungry

A very important feature about placeholders is that they require 
feeding. We need to feed them some value everytime we start a new 
session. If we try to run through a graph, without feeding values 
to placeholders, then we get an error message such as:

    InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape dim { size: 3 } dim { size: 4 }

See for yourself, run the following code:

In [ ]:
# ------------------------------------------------
#                                    Build a graph
# ------------------------------------------------
graph = tf.Graph()
with graph.as_default():
    a = tf.placeholder(tf.float32, shape=(3, 4))
    b = a * 2
    
# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as session:
    # run, but do not feed `a` explicitly
    output = session.run(b)
    print(output)

In order to feed **`a`** we can make use of the `feed_dict` argument 
in the `session.run()` method. The argument takes a dictionary, with 
the key being the placeholder objects you want to feed, and the 
values of the dictionary are the values you want to feed to the 
corresponding placceholder. 

If we now feed **`a`** a value, we get it to run properly wihtout errors. 

In [4]:
# Create an array to be fed to `a`
input_array = np.ones((3,4))

# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as session:
    # run, but also feed `a` a value
    output = session.run(b, feed_dict={a: input_array})
    print(output)

[[ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]]


### TODO: add a little discussion about feeding values and how it can technically be used for constant and variable tensors too, but that it is best practice to use it on placeholders.

## But I'm a fussy eater! 
Placeholders will only allow you to feed them values that match some particular shape and size. When we declared the placeholder, we told it that we want a 3 by 4 tensor, composed of floats. If we try to feed it a 10 by 4 tensor instead we get an error message such as: 

    ValueError: Cannot feed value of shape (10, 4) for Tensor u'Placeholder:0', which has shape (Dimension(3), Dimension(4))

Run the following code to see for yourself: 

In [ ]:
# Create an array to be fed to `a`
input_array = np.ones((10,4))

# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as session:
    # feed `a` a value that is not 3 by 4
    output = session.run(b, feed_dict={a: input_array})
    print(output)

## Flexibile diet

We may want the placeholder to be a little less fussy about what we 
feed it. For example when we are creating a neural network, the 
inputs may have numbers of samples. Our training, validation, 
and test sets usually wont have the same number of samples, and 
when we use it in production, we will also want to make 
predictions on different amounts of data, sometimes we may only 
want to make a prediction on a single sample. But, in all cases 
there are still similarities in the data, We will want all the 
data to have at least one of the dimensions to be the same size. 

With Placeholders, we can constrain the size along certain 
dimensions, and be completely flexible along other dimensions. 
In order to do so, we use the `shape` argument and place a 
`None` value for the dimensions we want to be flexible with, 
and an integer value for the dimensions we want to constrain.

In the example below we make the first axis (number of rows) 
flexible, and contrain the second axis (number of columns) 
to 4. We feed it a 5 by 4 tensor: 

In [6]:
# Create a 5 by 4 array to be fed to `a`
input_array = np.ones((5,4))

# ------------------------------------------------
#                                    Build a graph
# ------------------------------------------------
graph = tf.Graph()
with graph.as_default():
    # Create a placceholder that is flaxible along the first 
    # dimension and fixed along the second dimension. 
    a = tf.placeholder(tf.float32, shape=(None, 4))
    b = a * 2
    
# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as session:
    output = session.run(b, feed_dict={a: input_array})
    print(output)

[[ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]]


Now we test it with a 12 by 4 tensor

In [7]:
# Create a 12 by 4 array to be fed to `a`
input_array = np.ones((12,4))

# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as session:
    output = session.run(b, feed_dict={a: input_array})
    print(output)

[[ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]]


But if we test it on a tensor whose second axis is not 4, we get an error message like this: 

    ValueError: Cannot feed value of shape (12, 5) for Tensor u'Placeholder:0', which has shape (Dimension(None), Dimension(4))
